In [1]:
import torch

from tqdm import tqdm
from datasets import load_dataset
from sklearn.metrics import matthews_corrcoef
from transformers import RobertaTokenizer, RobertaForMaskedLM

/mnt/c/SAPDevelop/reposp/lit/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'sklearn'

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [3]:
sick = load_dataset("sick", split="validation")

Found cached dataset sick (/home/imger/.cache/huggingface/datasets/sick/default/0.0.0/c6b3b0b44eb84b134851396d6d464e5cb8f026960519d640e087fe33472626db)


In [65]:
MASK = tokenizer("<mask>", add_special_tokens=False)["input_ids"][0]
ENTAILMENT_IDS = tokenizer.convert_tokens_to_ids(["as", "because", "so"])
NEUTRAL_IDS = tokenizer.convert_tokens_to_ids(["and", "also", "or"])
CONTRADICTION_IDS = tokenizer.convert_tokens_to_ids(["but", "although", "still"])

def zero_shot_classify(premise, hypothesis):
    hypothesis = hypothesis.split(" ")
    hypothesis[0] = hypothesis[0].lower()
    hypothesis = " ".join(hypothesis)
    combined = f"{premise} <mask> {hypothesis}"
    tokenized, attention_mask = tokenizer(combined, return_tensors="pt", add_special_tokens=True).values()

    logits = model(tokenized, attention_mask=attention_mask)["logits"]
    mask_idx = tokenized[0].tolist().index(MASK)
    entailment_score = sum(logits[0, mask_idx, ENTAILMENT_IDS])
    neutral_score = sum(logits[0, mask_idx, NEUTRAL_IDS])
    contradiction_score = sum(logits[0, mask_idx, CONTRADICTION_IDS])

    max_score = max(entailment_score, neutral_score, contradiction_score)
    if entailment_score == max_score:
        return 0
    if neutral_score == max_score:
        return 1
    if contradiction_score == max_score:
        return 2
    assert False, "This should never be reached"

In [66]:
true, predicted = [], []
for d in tqdm(sick):
    true.append(d["label"])
    predicted.append(zero_shot_classify(d["sentence_A"], d["sentence_B"]))

100%|██████████| 495/495 [00:32<00:00, 15.27it/s]


ModuleNotFoundError: No module named 'sklearn'